In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Load the csv exported in Part I to a DataFrame

weather_csv_file = "../WeatherPy/output_data/city_weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hirado,33.3597,129.5531,75.20,35,0,4.61,JP,1617257768
1,rawson,-43.3002,-65.1023,59.16,87,100,7.20,AR,1617257769
2,cape town,-33.9258,18.4232,64.00,72,20,14.97,ZA,1617257553
3,tasiilaq,65.6145,-37.6368,26.60,54,40,4.61,GL,1617257769
4,mar del plata,-38.0023,-57.5575,66.00,100,96,6.31,AR,1617257736


In [3]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

humidity = weather_df["Humidity"].astype(float)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_df[["Lat", "Lng"]].astype(float)

# Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:

# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values.
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hirado,33.3597,129.5531,75.20,35,0,4.61,JP,1617257768
31,maceio,-9.6658,-35.7353,75.20,94,0,3.44,BR,1617257780
183,sakaraha,-22.9000,44.5333,74.95,57,0,3.87,MG,1617257843
413,bodden town,19.2833,-81.2500,78.01,1,0,1.99,KY,1617257948
447,bam,29.1060,58.3570,76.24,8,0,5.26,IR,1617257964
504,najran,17.4924,44.1277,78.80,17,0,6.91,SA,1617257993


In [5]:
#Store into variable named hotel_df.
hotel_df = filtered_weather_df

#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for i, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
    print(f"Retrieving Results for Index {i}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[i, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: hirado.
Closest hotel is Samson Hotel.
------------
Retrieving Results for Index 31: maceio.
Closest hotel is Poutur Pousada.
------------
Retrieving Results for Index 183: sakaraha.
Closest hotel is Le Relais.
------------
Retrieving Results for Index 413: bodden town.
Closest hotel is Will T Place.
------------
Retrieving Results for Index 447: bam.
Closest hotel is Bam Tourist Hotel.
------------
Retrieving Results for Index 504: najran.
Closest hotel is محمد هادي ال فطيح.
------------


In [6]:
#Add a "Hotel Name" column to the DataFrame.
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,hirado,33.3597,129.5531,75.20,35,0,4.61,JP,1617257768,Samson Hotel
31,maceio,-9.6658,-35.7353,75.20,94,0,3.44,BR,1617257780,Poutur Pousada
183,sakaraha,-22.9000,44.5333,74.95,57,0,3.87,MG,1617257843,Le Relais
413,bodden town,19.2833,-81.2500,78.01,1,0,1.99,KY,1617257948,Will T Place
447,bam,29.1060,58.3570,76.24,8,0,5.26,IR,1617257964,Bam Tourist Hotel
504,najran,17.4924,44.1277,78.80,17,0,6.91,SA,1617257993,محمد هادي ال فطيح


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
#Plot markers on top of the heatmap.
markers = gmaps.marker_layer(locations)

#Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#For Part II, you must include a screenshot of the heatmap you create and include it in your submission.